In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
np.random.seed(1)
categorías=['Guadalajara','Zapopan','Ciudad de México','Monterrey','Morelia','Mexicali']
ponderación=[1/4,1/5,1/4,7/40,1/20,3/40]
limites=[(3500,6000),(3000,12000),(4000,18000),(3000,6000),(2000,5000),(3500,5000)]
ciudad=np.random.choice(categorías,size=20,p=ponderación)
rentas=[]
for c in ciudad:
  rentas.append(np.random.randint(*limites[categorías.index(c)]))
df=pd.DataFrame()
df['Rentas']=rentas
df['Ciudad']=ciudad
df.head()

,Rentas,Ciudad
0,9285,Zapopan
1,5025,Monterrey
2,4531,Guadalajara
3,7415,Zapopan
4,4837,Guadalajara


#Codificación

##One-Hot

In [3]:
df['Ciudad'].unique()

array(['Zapopan', 'Monterrey', 'Guadalajara', 'Ciudad de México',
       'Morelia'], dtype=object)

In [4]:
df['Ciudad']=='Zapopan'

0      True
1     False
2     False
3      True
4     False
5     False
6     False
7      True
8      True
9     False
10     True
11    False
12    False
13    False
14    False
15    False
16     True
17    False
18    False
19    False
Name: Ciudad, dtype: bool

In [5]:
#Asigna un 1 si es una asignación
OH = pd.get_dummies(df,prefix='',prefix_sep='',columns=['Ciudad'])
OH

,Rentas,Ciudad de México,Guadalajara,Monterrey,Morelia,Zapopan
0,9285,0,0,0,0,1
1,5025,0,0,1,0,0
2,4531,0,1,0,0,0
3,7415,0,0,0,0,1
4,4837,0,1,0,0,0
5,5420,0,1,0,0,0
6,3816,0,1,0,0,0
7,11920,0,0,0,0,1
8,9771,0,0,0,0,1
9,4431,1,0,0,0,0


## Codificación ficticia

In [6]:
#Eliminar la primer columna de One-Hot
#Columns = ['Ciudad'] es para especificar la variable categórica que se quiere trabajar, en este caso solo hay una por lo que no es necesario
F = pd.get_dummies(df,prefix='',prefix_sep='',columns=['Ciudad'],drop_first=True)
F

,Rentas,Guadalajara,Monterrey,Morelia,Zapopan
0,9285,0,0,0,1
1,5025,0,1,0,0
2,4531,1,0,0,0
3,7415,0,0,0,1
4,4837,1,0,0,0
5,5420,1,0,0,0
6,3816,1,0,0,0
7,11920,0,0,0,1
8,9771,0,0,0,1
9,4431,0,0,0,0


In [7]:
# Qué categoría eliminó
CR,=set(df.Ciudad.unique())-set(F.columns)
CR

'Ciudad de México'

In [8]:
#Escogemos una categoría para eliminar (categoría de referencia)
CR = 'Monterrey'
F= pd.get_dummies(df,prefix='',prefix_sep='')
F.head()

,Rentas,Ciudad de México,Guadalajara,Monterrey,Morelia,Zapopan
0,9285,0,0,0,0,1
1,5025,0,0,1,0,0
2,4531,0,1,0,0,0
3,7415,0,0,0,0,1
4,4837,0,1,0,0,0


In [9]:
F.drop(columns=CR,inplace=True)
F.head()

,Rentas,Ciudad de México,Guadalajara,Morelia,Zapopan
0,9285,0,0,0,1
1,5025,0,0,0,0
2,4531,0,1,0,0
3,7415,0,0,0,1
4,4837,0,1,0,0


## Codificación de efectos

In [10]:
E = F.copy()
E.head()

,Rentas,Ciudad de México,Guadalajara,Morelia,Zapopan
0,9285,0,0,0,1
1,5025,0,0,0,0
2,4531,0,1,0,0
3,7415,0,0,0,1
4,4837,0,1,0,0


In [11]:
#Identificación de las observaciones
idx_CR=np.where(df['Ciudad']==CR)[0]
idx_CR

array([1])

In [12]:
E=E.astype('int')

In [13]:
E.iloc[idx_CR,1:]=-1
E.head()

,Rentas,Ciudad de México,Guadalajara,Morelia,Zapopan
0,9285,0,0,0,1
1,5025,-1,-1,-1,-1
2,4531,0,1,0,0
3,7415,0,0,0,1
4,4837,0,1,0,0


## Regresión

In [14]:
lin_OH=LinearRegression() #Regresión lineal para la codificación One-Hot
lin_F=LinearRegression() #Regresión lineal para la codificación ficticia
lin_E=LinearRegression() #Regresión lineal para la codificación de efectos

In [15]:
#Entrenamiento
lin_OH.fit(OH.drop(columns='Rentas'),OH.Rentas)
lin_F.fit(F.drop(columns='Rentas'),F.Rentas)
lin_E.fit(E.drop(columns='Rentas'),E.Rentas)

LinearRegression()

In [16]:
#One-Hot
lin_OH.coef_

array([ 4525.88333333, -1799.11666667, -1357.36666667, -3287.36666667,
        1917.96666667])

In [17]:
#Promedio de los promedios
lin_OH.intercept_

6382.366666666667

In [18]:
#Promedios para cada ciudad
promedios=df.groupby('Ciudad').mean()
promedios

,Rentas
Ciudad,
Ciudad de México,10908.250000
Guadalajara,4583.250000
Monterrey,5025.000000
Morelia,3095.000000
Zapopan,8300.333333


##Interpretación de la regresión lineal para la codificación de One-Shot

In [19]:
#La intersección es el promedio de 'promedios'
promedios.mean()-lin_OH.intercept_

Rentas    0.0
dtype: float64

In [20]:
#Los coeficientes son la diferencia entre los promedios y la intersección
coef= promedios-lin_OH.intercept_
coef

,Rentas
Ciudad,
Ciudad de México,4525.883333
Guadalajara,-1799.116667
Monterrey,-1357.366667
Morelia,-3287.366667
Zapopan,1917.966667


In [21]:
lin_OH.coef_

array([ 4525.88333333, -1799.11666667, -1357.36666667, -3287.36666667,
        1917.96666667])

##Interpretación de la regresión lineal para la codificación de Ficticia

In [25]:
#En la codificación ficticia, la intersección es el promedio de rentas de la categoría de referencia
lin_F.intercept_

5024.999999999998

In [26]:
#En la refresión lineal ficticia, los coeficientes son la renta de cada ciudad menos la intersección
lin_F.coef_

array([ 5883.25      ,  -441.75      , -1930.        ,  3275.33333333])

In [27]:
promedios

,Rentas
Ciudad,
Ciudad de México,10908.250000
Guadalajara,4583.250000
Monterrey,5025.000000
Morelia,3095.000000
Zapopan,8300.333333


In [28]:
promedios-promedios.loc[CR]

,Rentas
Ciudad,
Ciudad de México,5883.250000
Guadalajara,-441.750000
Monterrey,0.000000
Morelia,-1930.000000
Zapopan,3275.333333


##Interpretación de la regresión lineal para la codificación de Efectos

In [29]:
#La intersección es el promedio de los promedios de la renta en cada ciudad
lin_E.intercept_,lin_OH.intercept_

(6382.366666666666, 6382.366666666667)

In [31]:
#La renta de cada ciudad es la suma entre la intersección y los coeficientes
lin_E.coef_,lin_OH.coef_

(array([ 4525.88333333, -1799.11666667, -3287.36666667,  1917.96666667]),
 array([ 4525.88333333, -1799.11666667, -1357.36666667, -3287.36666667,
         1917.96666667]))

In [32]:
E.head()

,Rentas,Ciudad de México,Guadalajara,Morelia,Zapopan
0,9285,0,0,0,1
1,5025,-1,-1,-1,-1
2,4531,0,1,0,0
3,7415,0,0,0,1
4,4837,0,1,0,0


In [33]:
df.head()

,Rentas,Ciudad
0,9285,Zapopan
1,5025,Monterrey
2,4531,Guadalajara
3,7415,Zapopan
4,4837,Guadalajara


Para obtener el promedio de la categoría de referencia, restamos todos los coeficientes de la inersección (el promedio de los promedios)

In [38]:
#Ambos valores deben de ser iguales
lin_E.intercept_-lin_E.coef_.sum(), promedios.loc[CR]

(5024.9999999999945,
 Rentas    5025.0
 Name: Monterrey, dtype: float64)